还有些其他的提示词技巧，这里举三个例子

- 零样本提示
- 少样本提示
- 思维链(CoT)提示词

In [ ]:
### 加载模型：Qwen2.5-0.5B-Instruct模型
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_or_path = '/your/path/of/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map='auto', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.81it/s]


In [3]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

## 零样本提示

零样本提示的定义是不提供任何示例，直接让模型完成任务，依赖模型自身的知识和推理能力。

In [11]:
prompt_1="""请列出红色的三种水果，仅输出结果"""

In [12]:
print(inference(model,tokenizer,prompt_1))


{'response': '苹果、樱桃、草莓', 'generated_tokens_count': 6}


## 少样本提示

少样本提示（Few-shot Prompting）是指在提示词中提供少量示例（通常 1-5 个），让模型通过模仿示例的模式来完成任务。它比零样本提示更 “具象”，通过示例明确任务规则或格式，帮助模型更精准地理解需求，尤其适合需要特定格式、逻辑或风格的任务。

In [13]:
prompt_2="""
### 问题
请你根据输入的内容，判断其情绪属于“积极”、“消极”、“中立”的哪一个标签？仅输出标签内容

### 示例
输入：我今天心情很好，工作也很顺利。
输出：积极

输入：我对这部电影的评价很差，觉得浪费时间。
输出：消极

## 用户输入
输入：我对这次旅行的体验还可以，但没有特别惊喜。
输出：
"""

In [14]:
print(inference(model,tokenizer,prompt_2))

{'response': '中立', 'generated_tokens_count': 3}


## 思维链（CoT）提示
思维链提示是通过引导模型 一步步说出推理过程来解决问题的方式。它不直接要求答案，而是先让模型像人一样 “思考”—— 拆解问题、分析逻辑、逐步推导，最终得出结论。这种方式能帮助模型处理需要多步推理的任务（如数学题、逻辑题），减少因跳跃性思维导致的错误。

In [15]:
prompt_3="""我今年6岁，是妹妹的两倍。那么在我过80岁生日的时候，我的妹妹多少岁？"""
prompt_4="""我今年6岁，是妹妹的两倍。那么在我过80岁生日的时候，我的妹妹多少岁？让我们一步一步思考"""

In [21]:
print(inference(model,tokenizer,prompt_3))
print(inference(model,tokenizer,prompt_4))

{'response': '如果你现在6岁，而你妹妹是你年龄的一半，那么你现在妹妹应该是4岁。\n\n你们年龄差2岁。无论你长大到多大，你和你妹妹之间的年龄差始终不变。\n\n所以，当你80岁的时候，你的妹妹将会是80 - 2 = 78岁。', 'generated_tokens_count': 67}
{'response': '好的，我们来一步步分析这个问题。\n\n1. **确定你的年龄**：\n   - 你现在6岁。\n\n2. **确定妹妹的年龄**：\n   - 根据题目，你是妹妹的两倍大。\n   - 因此，妹妹的年龄为 \\( \\frac{6}{2} = 3 \\) 岁。\n\n3. **计算到80岁时你们各自会多大**：\n   - 当你80岁的时候，从现在起过了 \\( 80 - 6 = 74 \\) 年。\n   - 同样地，你的妹妹在这74年内也会增长74年。\n   - 所以，到你80岁的时候，你的妹妹将会是 \\( 3 + 74 = 77 \\) 岁。\n\n综上所述，当你80岁生日的时候，你的妹妹将会是77岁。', 'generated_tokens_count': 190}


不过要注意，并不是所有的问题都适合用该方法，很显然，当你遇到难以解决的数学问题的时候，CoT会有奇效。可如果你仅仅只是问一个很简单的问题，比如“中国的首都在哪？”，让模型一步一步思考显然是一个很浪费时间的行为。
因此可以得出结论，当需要保证结果的准确度、需要解决较为复杂的数学、推理问题的时候，思维链提示是一个好的方法，其他情况可以酌情考虑。